In [ ]:
!pip install transformers

import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Parameters
max_len = 128
batch_size = 16
epochs = 3
model_name = 'bert-base-uncased'

# Sample data loading (Replace with your actual data loading)
# df = pd.read_csv('your_dataset.csv')

# Load BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Assuming binary classification

# Tokenization
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True, max_length=max_len)

# Encode the labels
labels = df['label'].values  # Assuming binary labels

# Prepare the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], labels, test_size=0.2, random_state=42)

# Tokenize datasets
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=max_len)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True, max_length=max_len)

# Convert to PyTorch dataset
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=lambda p: {
        'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        'precision': precision_recall_fscore_support(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')[0],
        'recall': precision_recall_fscore_support(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')[1],
        'f1': precision_recall_fscore_support(p.label_ids, np.argmax(p.predictions, axis=1), average='binary')[2],
    }
)

# Train and evaluate
trainer.train()
eval_results = trainer.evaluate()

print(f"Evaluation Results: {eval_results}")

# If you just want to use a pre-trained model without training
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Example usage
example_texts = ["I love this!", "I hate it."]
predictions = classifier(example_texts)
print(predictions)
